In [1]:
import pandas as pd
import numpy as np
import os
from scipy.stats import fisher_exact
from statsmodels.stats.multitest import multipletests
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [2]:
# path
human_goslim_up_path = '../data/human_annotation/human_annotation_up.tsv'
rice_goslim_up_path = '../data/rice_annotation/rice_annotation_up.tsv'

human_goslim_all_path = '../data/biomart_goslim/biomart_human_goslim_R110.tsv'
rice_goslim_all_path = '../data/biomart_goslim/biomart_rice_goslim_R56.tsv'

In [3]:
human_goslim_up = pd.read_csv(human_goslim_up_path, sep='\t')
human_goslim_up.drop(columns=['Unnamed: 0'], inplace=True)
human_goslim_up

,Gene stable ID,Chromosome/scaffold name,Gene start(bp),Gene end(bp),GOSlim GOA Accession(s),GOSlim GOA Description,converted_alias,HN-score(HN5),name
0,ENSG00000177606,1,58776845,58784048,GO:0043226,organelle,ENSG00000177606,63,JUN
1,ENSG00000177606,1,58776845,58784048,GO:0005634,nucleus,ENSG00000177606,63,JUN
2,ENSG00000177606,1,58776845,58784048,GO:0003723,RNA binding,ENSG00000177606,63,JUN
3,ENSG00000177606,1,58776845,58784048,GO:0005654,nucleoplasm,ENSG00000177606,63,JUN
4,ENSG00000177606,1,58776845,58784048,GO:0006351,DNA-templated transcription,ENSG00000177606,63,JUN
...,...,...,...,...,...,...,...,...,...
4843,ENSG00000130822,X,153669733,153689010,GO:0140096,"catalytic activity, acting on a protein",ENSG00000130822,64,PNCK
4844,ENSG00000130822,X,153669733,153689010,GO:0036211,protein modification process,ENSG00000130822,64,PNCK
4845,ENSG00000130822,X,153669733,153689010,GO:0003824,catalytic activity,ENSG00000130822,64,PNCK
4846,ENSG00000130822,X,153669733,153689010,GO:0016740,transferase activity,ENSG00000130822,64,PNCK


In [4]:
rice_goslim_up = pd.read_csv(rice_goslim_up_path, sep='\t')
rice_goslim_up.drop(columns=['Unnamed: 0'], inplace=True)
rice_goslim_up

,Gene stable ID,Chromosome/scaffold name,Gene start(bp),Gene end(bp),GOSlim GOA Accession(s),GOSlim GOA Description,GENEID,HN-score(HN5)
0,Os01g0105800,1,306871,308842,GO:0008150,biological_process,Os01g0105800,53
1,Os01g0105800,1,306871,308842,GO:0008152,metabolic process,Os01g0105800,53
2,Os01g0105800,1,306871,308842,GO:0019538,protein metabolic process,Os01g0105800,53
3,Os01g0105800,1,306871,308842,GO:0009987,cellular process,Os01g0105800,53
4,Os01g0105800,1,306871,308842,GO:0016043,cellular component organization,Os01g0105800,53
...,...,...,...,...,...,...,...,...
2930,Os12g0600100,12,25242398,25246733,GO:0003674,molecular_function,Os12g0600100,61
2931,Os12g0600100,12,25242398,25246733,GO:0005515,protein binding,Os12g0600100,61
2932,Os12g0600100,12,25242398,25246733,GO:0005488,binding,Os12g0600100,61
2933,Os12g0600100,12,25242398,25246733,GO:0005575,cellular_component,Os12g0600100,61


In [5]:
human_goslim_all = pd.read_csv(human_goslim_all_path, sep='\t')
human_goslim_all

/tmp/ipykernel_8544/3662585687.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  human_goslim_all = pd.read_csv(human_goslim_all_path, sep='\t')


,Gene stable ID,Chromosome/scaffold name,start2 (bp),end2 (bp),GOSlim GOA Accession(s),GOSlim GOA Description
0,ENSG00000243485,1,29554,31109,GO:0031047,gene silencing by RNA
1,ENSG00000284332,1,30366,30503,GO:0031047,gene silencing by RNA
2,ENSG00000186092,1,65419,71585,GO:0023052,signaling
3,ENSG00000186092,1,65419,71585,GO:0060089,molecular transducer activity
4,ENSG00000186092,1,65419,71585,GO:0005886,plasma membrane
...,...,...,...,...,...,...
581396,ENSG00000292372,Y,57207346,57212230,GO:0007010,cytoskeleton organization
581397,ENSG00000292372,Y,57207346,57212230,GO:0008092,cytoskeletal protein binding
581398,ENSG00000292372,Y,57207346,57212230,GO:0031410,cytoplasmic vesicle
581399,ENSG00000292372,Y,57207346,57212230,GO:0043226,organelle


In [6]:
rice_goslim_all = pd.read_csv(rice_goslim_all_path, sep='\t')
rice_goslim_all

/tmp/ipykernel_8544/1087250297.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  rice_goslim_all = pd.read_csv(rice_goslim_all_path, sep='\t')


,Gene stable ID,Chromosome/scaffold name,start1 (bp),end1 (bp),GOSlim GOA Accession(s),GOSlim GOA Description
0,Os01g0100100,1,2983,10815,GO:0006810,transport
1,Os01g0100100,1,2983,10815,GO:0008150,biological_process
2,Os01g0100100,1,2983,10815,GO:0009987,cellular process
3,Os01g0100100,1,2983,10815,GO:0003674,molecular_function
4,Os01g0100100,1,2983,10815,GO:0030234,enzyme regulator activity
...,...,...,...,...,...,...
258565,gene-rps19,Pt,134200,134481,GO:0003674,molecular_function
258566,gene-rps19,Pt,134200,134481,GO:0005198,structural molecule activity
258567,gene-rps19,Pt,134200,134481,GO:0005488,binding
258568,gene-rps19,Pt,134200,134481,GO:0003723,RNA binding


In [16]:
def get_go_term_counts(dataframe, filename):
    output_dir = './counts/'
    os.makedirs(output_dir, exist_ok=True)

    # GOSlim GOA Accession(s) と GOSlim GOA Description のペアについてカウント
    counts_series = dataframe.groupby(['GOSlim GOA Accession(s)', 'GOSlim GOA Description']).size()
    counts_df = counts_series.reset_index(name='Count')

    # カウント結果を TSV として保存
    counts_df.to_csv(f'./counts/{filename}', sep='\t', index=False)
    print(f'Saved GO term counts to {filename}')
    return counts_df

In [17]:
human_goslim_up_counts = get_go_term_counts(human_goslim_up, "human_goslim_up_counts.tsv")
rice_goslim_up_counts = get_go_term_counts(rice_goslim_up, "rice_goslim_up_counts.tsv")
human_goslim_all_counts = get_go_term_counts(human_goslim_all, "human_goslim_all_counts.tsv")
rice_goslim_all_counts = get_go_term_counts(rice_goslim_all, "rice_goslim_all_counts.tsv")
print(type(human_goslim_up_counts))
print(type(human_goslim_all_counts))
print(type(rice_goslim_up_counts))
print(type(rice_goslim_all_counts))

Saved GO term counts to human_goslim_up_counts.tsv
Saved GO term counts to rice_goslim_up_counts.tsv
Saved GO term counts to human_goslim_all_counts.tsv
Saved GO term counts to rice_goslim_all_counts.tsv
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


#### Enrichment analysis

In [15]:
def perform_goslim_enrichment(target_df, background_df, filename):
    """
    Perform GO_Slim enrichment analysis comparing target GO term counts against background counts.

    Parameters:
    target_df (pandas.DataFrame): DataFrame containing counts of GO terms in the target set.
    background_df (pandas.DataFrame): DataFrame containing counts of GO terms in the background set.
    filename (str): The filename for saving the enrichment results.

    Returns:
    pd.DataFrame: Dataframe containing GO terms, their fold enrichment, raw p-values, and FDR-adjusted p-values.
    """
    # 'Count' 列を数値型に変換
    target_df['Count'] = pd.to_numeric(target_df['Count'], errors='coerce').fillna(0)
    background_df['Count'] = pd.to_numeric(background_df['Count'], errors='coerce').fillna(0)
    
    enrichment_outputdir = './enrichment_result/'
    os.makedirs(enrichment_outputdir, exist_ok=True)

    results = []
    for index, row in target_df.iterrows():
        term = row['GO Term']
        target_freq = row['Count']
        # 背景のカウントを取得
        background_row = background_df[background_df['GO Term'] == term]
        background_freq = background_row['Count'].iloc[0] if not background_row.empty else 0
        total_target = target_df['Count'].sum()
        total_background = background_df['Count'].sum()

        # エンリッチメントスコアとp値の計算
        oddsratio, pvalue = fisher_exact([[target_freq, total_target - target_freq],
                                          [background_freq, total_background - background_freq]],
                                         alternative='greater')
        fold_enrichment = oddsratio if background_freq > 0 else 0  # 除数が0の場合の処理
        results.append((term, fold_enrichment, pvalue))
    
    # 結果をデータフレームに変換
    terms, fold_enrichments, pvalues = zip(*results)
    _, pvals_corrected, _, _ = multipletests(pvalues, method='fdr_bh')

    enrichment_df = pd.DataFrame({
        'GO_slim': terms,
        'Fold Enrichment': fold_enrichments,
        'p-value': pvalues,
        'FDR': pvals_corrected
    })

    enrichment_df.to_csv(f'{enrichment_outputdir}/{filename}', sep='\t', index=False)
    print(f'Enrichment analysis results saved to {enrichment_outputdir}/{filename}')
    return enrichment_df

# エンリッチメント解析を実行する前に、DataFrameの 'Count' 列が数値であることを確認
human_goslim_up_counts['Count'] = pd.to_numeric(human_goslim_up_counts['Count'], errors='coerce').fillna(0)
rice_goslim_up_counts['Count'] = pd.to_numeric(rice_goslim_up_counts['Count'], errors='coerce').fillna(0)

# DataFrameを関数に渡す
human_enrichment = perform_goslim_enrichment(human_goslim_up_counts, human_goslim_all_counts, "human_enrichment.tsv")


Enrichment analysis results saved to ./enrichment_result//human_enrichment.tsv
